In [1]:
!pip install pyngrok==4.1.1 flask-ngrok
!pip install bootstrap-flask Flask-WTF WTForms
!ngrok authtoken '29wnsR4eRexbsj5gEAQb8NEDHbD_6gRmkUqyv8CaLzc1jD9DB'

from flask import Flask, render_template_string, redirect, url_for
from flask_bootstrap import Bootstrap4
from flask_wtf import FlaskForm
from wtforms import StringField, RadioField, IntegerField, FloatField, SubmitField
from wtforms.validators import DataRequired, InputRequired
from flask_ngrok import run_with_ngrok

from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

model = keras.models.load_model("/content/drive/Shared drives/ECS171 Group Project Team 11/best_model")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=fd92278563ae753e051f6be52ee221d1c1a9f0b7d49a76c03d32fa245b92b58d
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 6.7 MB/s 
     |████████████████████████████████| 136 kB 73.7 MB/s 
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Mounted at /content/drive


In [2]:
def process_input(i):
    df = pd.DataFrame(i)
    # Replace Binary Columns No with 0
    df.iloc[:,5:14] = df.iloc[:,5:14].replace('No', 0)
    df['HeartDisease'] = df['HeartDisease'].replace('No', 0)

    # Replace Binary Columns Yes with 1
    df.iloc[:,5:14] = df.iloc[:,5:14].replace('Yes', 1)
    df['HeartDisease'] = df['HeartDisease'].replace('Yes', 1)

    # Rename Sex column to Female
    df = df.rename({'Sex': 'Female'}, axis=1)

    # Replace Male with 0, Female with 1, and some typo corrections
    df = df.replace({
        'Female': {
            'Female': 1,
            'Male': 0
        },
        'Diabetic': {
            'No, borderline diabetes': 'No_borderline_diabetes',
            'Yes (during pregnancy)': 'Yes_during_pregnancy'
        },
        'Race': {
            'American Indian/Alaskan Native': 'American_Indian_or_Alaskan_Native'
        },
        'GenHealth': {
            'Very good': 'Very_good'
        }

    })

    df.head()

    df_ds = pd.read_csv("/content/drive/Shared drives/ECS171 Group Project Team 11/heart_2020_cleaned.csv")
    df_ds = df_ds[[
        'BMI',
        'PhysicalHealth',
        'MentalHealth',
        'SleepTime'
    ]]
    scaler = MinMaxScaler()
    scaler.fit(df_ds)
    df.iloc[:, 1:5] = scaler.transform(df.iloc[:, 1:5])

    columns = ['HeartDisease', 'BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime',
              'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Female',
              'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer',
              'AgeCategory__18-24', 'AgeCategory__25-29', 'AgeCategory__30-34',
              'AgeCategory__35-39', 'AgeCategory__40-44', 'AgeCategory__45-49',
              'AgeCategory__50-54', 'AgeCategory__55-59', 'AgeCategory__60-64',
              'AgeCategory__65-69', 'AgeCategory__70-74', 'AgeCategory__75-79',
              'AgeCategory__80 or older', 'Race__American_Indian_or_Alaskan_Native',
              'Race__Asian', 'Race__Black', 'Race__Hispanic', 'Race__Other',
              'Race__White', 'Diabetic__No', 'Diabetic__No_borderline_diabetes',
              'Diabetic__Yes', 'Diabetic__Yes_during_pregnancy',
              'GenHealth__Excellent', 'GenHealth__Fair', 'GenHealth__Good',
              'GenHealth__Poor', 'GenHealth__Very_good']

    for col in columns:
        parts = col.split('__')

        if len(parts) == 2:
            if df[parts[0]][0] == parts[1]:
                df[col] = [1]
            else:
                df[col] = [0]

    df = df.drop(columns=['HeartDisease', 'AgeCategory', 'Race', 'Diabetic', 'GenHealth'])

    return df

In [3]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'C2HWGVoMGfNTBsrYQg8EcMrdTimkZfAbssdlkfjl'
run_with_ngrok(app)  # Start ngrok when app is run
bootstrap = Bootstrap4(app)


class NameForm(FlaskForm):
    BMI = FloatField('BMI', validators=[InputRequired()])
    PhysicalHealth = IntegerField('PhysicalHealth', validators=[InputRequired()])
    MentalHealth = IntegerField('MentalHealth', validators=[InputRequired()])
    SleepTime = IntegerField('SleepTime', validators=[InputRequired()])
    Smoking = RadioField('Smoking', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    AlcoholDrinking = RadioField('AlcoholDrinking', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    Stroke = RadioField('Stroke', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    DiffWalking = RadioField('DiffWalking', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    Sex = RadioField('Sex', choices=[('Female','Female'),('Male','Male')], validators=[DataRequired()])
    PhysicalActivity = RadioField('PhysicalActivity', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    Asthma = RadioField('Asthma', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    KidneyDisease = RadioField('KidneyDisease', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    SkinCancer = RadioField('SkinCancer', choices=[('No','No'),('Yes','Yes')], validators=[DataRequired()])
    AgeCategory = RadioField('AgeCategory', choices=[('55-59','55-59'), ('80 or older','80 or older'), ('65-69','65-69'), ('75-79', '75-79'), ('40-44', '40-44'), ('70-74','70-74'), ('60-64','60-64'), ('50-54','50-54'), ('45-49','45-49'), ('18-24','18-24'), ('35-39','35-39'), ('30-34','30-34'), ('25-29','25-29')], validators=[DataRequired()])
    Race = RadioField('Race', choices=[('White','White'), ('Black','Black'), ('Asian','Asian'), ('American Indian/Alaskan Native','American Indian/Alaskan Native'), ('Other','Other'), ('Hispanic','Hispanic')], validators=[DataRequired()])
    Diabetic = RadioField('Diabetic', choices=[('Yes','Yes'), ('No','No'), ('No, borderline diabetes','No, borderline diabetes'), ('Yes (during pregnancy)','Yes (during pregnancy)')], validators=[DataRequired()])
    GenHealth = RadioField('GenHealth', choices=[('Very good','Very good'), ('Fair','Fair'), ('Good','Good'), ('Poor','Poor'), ('Excellent','Excellent')], validators=[DataRequired()])

    submit = SubmitField('Submit')

@app.route('/', methods=['GET', 'POST'])
def index():
    form = NameForm()
    message = ""
    if form.validate_on_submit():

        data = {
            'HeartDisease': None,
            'BMI' : [form.BMI.data],
            'PhysicalHealth' :[form.PhysicalHealth.data],
            'MentalHealth' :[form.MentalHealth.data],
            'SleepTime' :[form.SleepTime.data],
            'Smoking':[form.Smoking.data],
            'AlcoholDrinking': [form.AlcoholDrinking.data],
            'Stroke': [form.Stroke.data],
            'DiffWalking':[form.DiffWalking.data],
            'Sex' : [form.Sex.data],
            'PhysicalActivity': [form.PhysicalActivity.data],
            'Asthma':[form.Asthma.data],
            'KidneyDisease':[form.KidneyDisease.data],
            'SkinCancer':[form.SkinCancer.data],
            'AgeCategory': [form.AgeCategory.data],
            'Race':[form.Race.data],
            'Diabetic' :[form.Diabetic.data],
            'GenHealth' : [form.GenHealth.data]
        }

        df = process_input(data)

        message = f'Probability of Heart Disease: {model.predict(df)[0][0]}'

    return render_template_string(
        """ 
{% from 'bootstrap4/form.html' import render_form %}
<html>
<head>
<!-- Bootstrap CSS -->
</head>
<body>

<h2>Heart Disease Prediction Model</h2>
{{ render_form(form) }}

<p class="pt-5"><strong>{{ message }}</strong></p>

<!-- Bootstrap JS -->
</body>
</html>
        """, form=form, message=message)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d014-34-125-144-194.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Jun/2022 17:59:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 17:59:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 17:59:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 17:59:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2022 17:59:49] "POST / HTTP/1.1" 200 -
